# Wektory słów

W ramach drugiej części ćwiczeń przyjrzymy się gęstej reprezentacji semantycznej, czyli wektorom słów (ang. *word embeddings*):

Ćwiczenia wykonamy przy użyciu znakomitej biblioteki [SpaCy](http://spacy.io). W ćwiczeniach wykorzystamy zarówno mode języka angielskiego, jak i języka polskiego. W celu dokładniejszego przyjrzenia się działaniu wektorów proponuję wykorzystać największy model językowy dla języka polskiego.

```bash
python -m spacy download pl_core_news_lg

pythom -m spacy validate
```

In [ ]:
import spacy

nlp = spacy.load('pl_core_news_lg')

allWords = [
    orth
    for orth in nlp.vocab.vectors 
    if nlp.vocab[orth].has_vector
]

## Wprowadzenie do wektorów słów

In [ ]:
nlp('król').vector

In [ ]:
words = ['król','królik','cesarz','kier','lew','Wojtuś','Sławomir']

for w in words:
    print(f"{w:>12}: {nlp('król').similarity(nlp(w))}")

Poniższa funkcja pozwoli nam na przeszukanie całego korpusu słów i znalezienie top-n najbardziej podobnych słów do danego wektora.

In [ ]:
from tqdm.notebook import tqdm

def find_similar_vectors(vec, topn=10, tabu=list(), only_lowercase=True):

    def _cosine(v1, v2): return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

    results = [
        w
        for w in tqdm(allWords)
        if nlp.vocab[w].text not in tabu
        and nlp.vocab[w].is_lower == only_lowercase
        and _cosine(nlp.vocab[w].vector, vec) > 0.5
    ]
    
    results.sort(key=lambda w: _cosine(nlp.vocab[w].vector, vec), reverse=True)

    for r in results[:topn]:
        print(f'{nlp.vocab[r].text:>15}: {_cosine(nlp.vocab[r].vector, vec):.8f}')

In [ ]:
v = nlp('król').vector

find_similar_vectors(v)

Wektory mogą posłużyć do znajdowania analogii między słowami:

$$ \mathit{król} - \mathit{mężczyzna} = \mathit{???} - \mathit{kobieta} $$

$$ \mathit{???} = \mathit{król} - \mathit{mężczyzna} + \mathit{kobieta} $$

In [ ]:
v = nlp('król').vector - nlp('mężczyzna').vector + nlp('kobieta').vector

find_similar_vectors(v, tabu=['król','mężczyzna','kobieta'])

### zadanie samodzielne

Wykonaj odpowiednie działania na wektorach słów aby znaleźć ukryte słowa:

- Niemcy - Berlin = Rosja - ?
- ? = czerwony + żółty
- tygrys - kot = wilk - ?
- jem - zjadłem = idę - ?

## Wykorzystanie wektorów słów

W poniższym przykładzie wykorzystamy technikę grupowania (ang. *clustering*) do automatycznego pogrupowania zdań pochodzących z [artykułu o Poznaniu](https://pl.wikipedia.org/wiki/Pozna%C5%84)

In [ ]:
from bs4 import BeautifulSoup
from wordfreq import zipf_frequency

import requests
import re

respond = requests.get("https://pl.wikipedia.org/wiki/Poznań")
soup = BeautifulSoup(respond.text, "html")
page = soup.find_all('p')

raw_text = ''.join([re.sub(r'\[\d+\]', '', paragraph.text) for paragraph in page])
raw_text = raw_text.replace(',', ' ').replace('(', ' ').replace(')', ' ')

In [ ]:
words = {
    word.lower(): nlp(word).vector
    for word in tqdm(set(raw_text.split()))
    if zipf_frequency(word, 'pl') < 5
    and zipf_frequency(word, 'pl') > 0
    and np.any(nlp(word).vector)
}

print(f"Liczba unikalnych słów w artykule o Poznaniu: {len(words)}")

word_vectors = np.vstack(list(words.values()))

In [ ]:
k = 5

model = KMeans(n_clusters=k, max_iter=1000)
model.fit(word_vectors)

In [ ]:
for i, c in enumerate(model.cluster_centers_):

    cluster_words = sorted(words.items(), key=lambda x: np.dot(x[1], c), reverse=True)
    print(f"Temat {i}: {[k for k,v in cluster_words][:5]}")

### zadanie samodzielne

Napisz funkcję, która dla podanego słowa znajdzie zbiór *k* najmniej podobnych słów. Opcjonalnie: postaraj się ograniczyć zbiór rozważanych słów tylko do słów które stanowią tę samą część mowy co podane słowo, tj. dla słowa "biały" poszukaj tylko najmniej podobnych przymiotników.

Wynik powinien wyglądać mniej więcej tak:

```python
find_dissimilar_words('dzień')
```

    vitara: -0.27556160
    elka: -0.26526517
    tesli: -0.26290190
    zippo: -0.25923923
    panasonica: -0.25569871
    durnia: -0.23886403
    demka: -0.23708199
    babola: -0.23590432
    bina: -0.23532979
    virago: -0.23311970

## Wektory zdań

W tym ćwiczeniu wykorzystamy dostarczony przez Google [Universal Sentence Encoder](https://tfhub.dev/google/universal-sentence-encoder/4). W celu przeprowadzenia ćwiczenia konieczne jest zainstalowanie wersji biblioteki `tensorflow` powyżej 1.15.

```bash
pip install "tensorflow>=1.15,<2.0"

pip install tensorflow_hub
```

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

import re, os, csv

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
with open('godzilla.srt') as f:
    phrases = f.readlines()

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

phrases = list(map(remove_tags, phrases))
phrases = [ phrase.rstrip() for phrase in phrases if len(phrase.split()) > 3 ]

annotations = [
    'we are going to die',
    'we will all die',
    "i don't want to die",
    'we will not survive'
]

print(f'Długość listy dialogowej filmu: {len(phrases)}')

In [ ]:
phrase_input_placeholder = tf.placeholder(tf.string, shape=(None))
annotation_input_placeholder = tf.placeholder(tf.string, shape=(None))

phrase_encoding = tf.nn.l2_normalize(embed(phrase_input_placeholder), dim=1)
annotation_encoding = tf.nn.l2_normalize(embed(annotation_input_placeholder), dim=1)

similarity_score = tf.matmul(phrase_encoding, annotation_encoding, adjoint_b=True)

similarity_threshold = 0.5

In [ ]:
with tf.Session() as session:
    
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    
    phrase_embeddings_, annotation_embeddings_, cosine_dist = session.run(
        [phrase_encoding, annotation_encoding, similarity_score],
        feed_dict = {
            phrase_input_placeholder: phrases,
            annotation_input_placeholder: annotations
        })
    
    for i, phrase_embedding in enumerate(phrase_embeddings_):
        for j, annotation_embedding in enumerate(annotation_embeddings_):
            if cosine_dist[i][j] > similarity_threshold:
                print(f'PHRASE: {phrases[i]:<40} ANNOTATION: {annotations[j]:<30} distance={cosine_dist[i][j]:.4f}')